# Интеграция CoreML RusTorch - Rust Kernel

Этот блокнот демонстрирует, как использовать CoreML с RusTorch.
Выполняется на ядре Rust.

## Проверка необходимых зависимостей и функций

In [ ]:
// Базовое использование RusTorch
extern crate rustorch;

use rustorch::tensor::Tensor;
use rustorch::gpu::DeviceType;

println!("Версия RusTorch: {}", env!("CARGO_PKG_VERSION"));
println!("Версия Rust: {}", env!("RUSTC_VERSION"));

## Проверка доступности CoreML

Проверить, доступен ли CoreML в текущей системе.

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::DeviceManager;
    
    let coreml_available = DeviceManager::is_coreml_available();
    println!("CoreML доступен: {}", coreml_available);
    
    if coreml_available {
        println!("🎉 CoreML доступен!");
        println!("Платформа: macOS");
        
        // Показать информацию об устройстве
        use rustorch::gpu::coreml::device_cache::DeviceCache;
        let cache = DeviceCache::global();
        cache.warmup();
        
        let stats = cache.get_stats();
        println!("Статистика кэша: {:?}", stats);
    } else {
        println!("⚠️ CoreML недоступен");
        println!("Пожалуйста, используйте CPU или другие GPU бэкенды");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("❌ Функции CoreML не включены");
    println!("Пожалуйста, соберите с --features coreml");
}

## Базовые операции с тензорами

In [ ]:
// Создать базовые тензоры
let a = Tensor::zeros(&[2, 3]);
let b = Tensor::ones(&[3, 2]);

println!("Форма тензора A: {:?}", a.shape());
println!("Форма тензора B: {:?}", b.shape());

// Базовое умножение матриц
let result = a.matmul(&b);
println!("Форма результата: {:?}", result.shape());
println!("Базовые операции с тензорами завершены");

## Операции с устройством CoreML

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::gpu::coreml::{CoreMLDevice, CoreMLBackend};
    use rustorch::backends::BackendConfig;
    
    // Попытаться создать устройство CoreML
    match CoreMLDevice::new(0) {
        Ok(device) => {
            println!("🖥️ Устройство CoreML успешно создано");
            println!("ID устройства: {}", device.id());
            println!("Доступно: {}", device.is_available());
            println!("Лимит памяти: {} МБ", device.memory_limit() / (1024 * 1024));
            
            // Создать конфигурацию бэкенда
            let config = BackendConfig::new()
                .with_caching(true)
                .with_max_cache_size(200)
                .with_profiling(true)
                .with_auto_fallback(true);
            
            println!("⚙️ Конфигурация бэкенда: {:?}", config);
            
            // Создать бэкенд CoreML
            match CoreMLBackend::new(device, config) {
                Ok(backend) => {
                    println!("🚀 Бэкенд CoreML инициализирован");
                    
                    // Получить статистику
                    let stats = backend.get_statistics();
                    println!("📊 Статистика бэкенда:");
                    println!("  Всего операций: {}", stats.total_operations);
                    println!("  Попадания в кэш: {}", stats.cache_hits);
                    println!("  Промахи кэша: {}", stats.cache_misses);
                    println!("  Операции отката: {}", stats.fallback_operations);
                    
                    // Создать тензоры на CoreML
                    let tensor_a = Tensor::randn(&[64, 64]).to_device(&backend);
                    let tensor_b = Tensor::randn(&[64, 64]).to_device(&backend);
                    
                    println!("📐 Тензоры созданы на устройстве CoreML");
                    
                    // Операция умножения матриц
                    let start = std::time::Instant::now();
                    let result = tensor_a.matmul(&tensor_b);
                    let duration = start.elapsed();
                    
                    println!("✅ Умножение матриц завершено");
                    println!("⏱️ Время выполнения: {:?}", duration);
                    println!("🎯 Форма результата: {:?}", result.shape());
                    
                    // Очистить кэш
                    backend.cleanup_cache();
                    println!("🧹 Кэш очищен");
                }
                Err(e) => println!("❌ Ошибка создания бэкенда CoreML: {:?}", e),
            }
        }
        Err(e) => {
            println!("❌ Ошибка создания устройства CoreML: {:?}", e);
            println!("CoreML может быть недоступен в этой системе");
        }
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Пропускаем операции CoreML - функции не включены");
}

## Сравнение производительности

In [ ]:
use std::time::Instant;

fn benchmark_operations() {
    let sizes = vec![(64, 64), (128, 128), (256, 256), (512, 512)];
    
    println!("🏁 Бенчмаркинг операций:");
    println!("Размер\t\tCPU (мс)\tПредпочтительное устройство");
    println!("-" * 55);
    
    for (rows, cols) in sizes {
        // Создать тензоры на CPU
        let a = Tensor::randn(&[rows, cols]);
        let b = Tensor::randn(&[cols, rows]);
        
        // Измерить время CPU
        let start = Instant::now();
        let _result = a.matmul(&b);
        let cpu_duration = start.elapsed();
        
        // Определить предпочтительное устройство
        let preferred_device = if rows * cols < 1000 {
            "CPU"
        } else if rows * cols < 10000 {
            "Metal GPU"
        } else {
            "CoreML"
        };
        
        println!("{}x{}\t\t{:.2}\t\t{}", 
                rows, cols, 
                cpu_duration.as_millis() as f64, 
                preferred_device);
    }
}

benchmark_operations();
println!("\n📝 Примечание: Выбор устройства основан на размере тензора и доступности");

## Умный выбор устройства

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::{DeviceManager, DeviceSelector};
    
    fn demonstrate_device_selection() {
        println!("🎯 Демонстрация умного выбора устройства:");
        
        let operations = vec![
            ("Маленькое умножение", vec![16, 16], "CPU"),
            ("2D свертка", vec![32, 3, 224, 224], "CoreML"),
            ("Преобразование матрицы", vec![128, 128], "Metal GPU"),
            ("Большая партийная операция", vec![512, 512], "CoreML"),
            ("Векторное вычисление", vec![1000], "CPU"),
        ];
        
        for (name, shape, preferred) in operations {
            println!("  {:<25} {:?} -> {}", name, shape, preferred);
            
            // Симулировать выбор на основе правил
            let tensor_size: usize = shape.iter().product();
            let selected_device = match tensor_size {
                size if size < 1000 => DeviceType::Cpu,
                size if size < 50000 => DeviceType::MetalGpu,
                _ => {
                    if DeviceManager::is_coreml_available() {
                        DeviceType::CoreML
                    } else {
                        DeviceType::MetalGpu
                    }
                }
            };
            
            println!("    -> Выбранное устройство: {:?}", selected_device);
        }
        
        println!("\n📝 Логика выбора:");
        println!("  • < 1K элементов: CPU (минимальные накладные расходы)");
        println!("  • 1K-50K элементов: Metal GPU (сбалансированный)");
        println!("  • > 50K элементов: CoreML (оптимизированный) или Metal GPU (откат)");
    }
    
    demonstrate_device_selection();
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Демонстрация выбора устройства пропущена - функции CoreML недоступны");
}

## Продвинутый пример: слой нейронной сети

In [ ]:
fn simulate_neural_layer() {
    println!("🧠 Симуляция слоя нейронной сети:");
    
    // Конфигурация слоя
    let batch_size = 32;
    let input_dim = 784;   // 28x28 MNIST
    let hidden_dim = 256;
    let output_dim = 10;   // 10 классов
    
    println!("📊 Конфигурация:");
    println!("  Размер партии: {}", batch_size);
    println!("  Входная размерность: {}", input_dim);
    println!("  Скрытая размерность: {}", hidden_dim);
    println!("  Выходная размерность: {}", output_dim);
    
    // Создать тензоры
    let input = Tensor::randn(&[batch_size, input_dim]);
    let weight1 = Tensor::randn(&[input_dim, hidden_dim]);
    let weight2 = Tensor::randn(&[hidden_dim, output_dim]);
    
    println!("\n🔄 Прямой проход:");
    
    // Симулированный прямой проход
    let start = Instant::now();
    
    // Слой 1: вход -> скрытый
    let hidden = input.matmul(&weight1);
    println!("  ✅ Вход -> Скрытый: {:?}", hidden.shape());
    
    // Функция активации ReLU (симулированная)
    let activated = hidden.relu();
    println!("  ✅ Активация ReLU применена");
    
    // Слой 2: скрытый -> выход
    let output = activated.matmul(&weight2);
    println!("  ✅ Скрытый -> Выход: {:?}", output.shape());
    
    let total_time = start.elapsed();
    
    println!("\n⏱️ Общее время прямого прохода: {:?}", total_time);
    println!("🚀 Расчетная производительность: {:.0} образцов/секунду", 
             (batch_size as f64) / total_time.as_secs_f64());
    
    println!("\n📝 В реальной реализации:");
    println!("  • Большие матрицы использовали бы CoreML");
    println!("  • Активации использовали бы Metal GPU");
    println!("  • Маленькие операции остались бы на CPU");
}

simulate_neural_layer();

## Обработка ошибок и откат

In [ ]:
fn demonstrate_fallback_behavior() {
    println!("🔄 Демонстрация поведения отката:");
    
    // Симулировать операцию, которая может не сработать на CoreML
    let complex_tensor = Tensor::randn(&[100, 100]);
    
    println!("🎯 Попытка операции CoreML...");
    
    // В реальной реализации это было бы:
    // match tensor.to_coreml() {
    //     Ok(coreml_tensor) => { /* использовать CoreML */ },
    //     Err(_) => { /* откат к Metal/CPU */ }
    // }
    
    let use_coreml = false; // Симулировать сбой CoreML
    
    if use_coreml {
        println!("✅ Операция CoreML успешна");
    } else {
        println!("⚠️ CoreML недоступен, используем откат");
        
        // Откат к Metal GPU
        let start = Instant::now();
        let result = complex_tensor.matmul(&complex_tensor);
        let fallback_time = start.elapsed();
        
        println!("✅ Операция отката завершена");
        println!("⏱️ Время отката: {:?}", fallback_time);
        println!("📐 Форма результата: {:?}", result.shape());
    }
    
    println!("\n📝 Стратегия отката:");
    println!("  1. Попробовать CoreML (лучшая производительность)");
    println!("  2. Откат к Metal GPU (хорошая совместимость)");
    println!("  3. Финальный откат к CPU (максимальная совместимость)");
}

demonstrate_fallback_behavior();

## Резюме и следующие шаги

In [ ]:
println!("📋 Резюме интеграции CoreML RusTorch (Rust Kernel):");
println!();
println!("✅ Продемонстрированные функции:");
println!("  • Проверка доступности CoreML");
println!("  • Создание и управление устройствами");
println!("  • Конфигурация бэкенда");
println!("  • Базовые операции с тензорами");
println!("  • Бенчмаркинг производительности");
println!("  • Умный выбор устройства");
println!("  • Поведение отката");
println!();
println!("🚧 Область разработки:");
println!("  • Полная реализация операций CoreML");
println!("  • Оптимизация передачи памяти");
println!("  • Расширенная поддержка типов тензоров");
println!("  • Детальное профилирование производительности");
println!("  • Интеграция с ML пайплайнами");
println!();
println!("🎯 Рекомендуемые следующие шаги:");
println!("  1. Тестировать с предобученными моделями CoreML");
println!("  2. Сравнительный бенчмарк с другими бэкендами");
println!("  3. Оптимизировать для конкретных случаев использования");
println!("  4. Развернуть в производственных приложениях");
println!();

#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    if rustorch::backends::DeviceManager::is_coreml_available() {
        println!("🎉 Все функции CoreML доступны для тестирования!");
    } else {
        println!("⚠️ CoreML включен, но недоступен в этой системе");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Соберите с функциями CoreML для полной функциональности");
}

println!("\n🚀 Готов к продвинутой разработке CoreML с RusTorch!");